In [3]:
!pip install xgboost
!pip install PuLP

In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from catboost import CatBoostRegressor, Pool,cv
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import pickle
import pulp
from pulp import value
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
df = pd.read_csv('merged_all_encoded2.csv')
df

,Match_Id,Season,Date,PlayOffs,Name,Position,Team,Opponent,Venue,TossWinner,...,Wickets_Roll_5_median,Econ_Roll_5_median,Dots_Roll_5_median,Fours_Roll_5_median,Sixes_Roll_5_median,Wd_Roll_5_median,Nb_Roll_5_median,Balls_bowled_Roll_5_median,Catches_Roll_5_median,total_points_Roll_5_median
0,335982,2008,2008-04-18,0,110,0,17,9,2,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,335982,2008,2008-04-18,0,223,0,17,9,2,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,335982,2008,2008-04-18,0,623,1,17,9,2,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,335982,2008,2008-04-18,0,628,1,17,9,2,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,335982,2008,2008-04-18,0,430,0,17,9,2,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19487,1312200,2022,2022-05-29,1,501,1,14,6,1,14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0
19488,1312200,2022,2022-05-29,1,252,1,14,6,1,14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,40.0
19489,1312200,2022,2022-05-29,1,633,1,14,6,1,14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0
19490,1312200,2022,2022-05-29,1,417,2,14,6,1,14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0


In [7]:
df.dropna(subset=['Runs_Roll_5_median'], inplace=True)
df.isnull().sum()

Match_Id                         0
Season                           0
Date                             0
PlayOffs                         0
Name                             0
Position                         0
Team                             0
Opponent                         0
Venue                            0
TossWinner                       0
TossDecision                     0
Runs                             0
Balls                            0
4s                               0
6s                               0
SR                               0
Maidens                          0
Runs_conceded                    0
Wickets                          0
Econ                             0
Dots                             0
Fours                            0
Sixes                            0
Wd                               0
Nb                               0
Balls_bowled                     0
Catches                          0
WinningTeam                      0
WonBy               

In [8]:
print(df[df['Margin'].isna()].iloc[:,:])
df.fillna(0, inplace=True)

       Match_Id  Season        Date  PlayOffs  Name  Position  Team  Opponent  \
1452     392190    2009  2009-04-23         0   640         0    14         9   
1453     392190    2009  2009-04-23         0    66         2     9        14   
1454     392190    2009  2009-04-23         0    26         2     9        14   
1455     392190    2009  2009-04-23         0    28         2     9        14   
1456     392190    2009  2009-04-23         0   193         0    14         9   
...         ...     ...         ...       ...   ...       ...   ...       ...   
17771   1216547    2020  2020-09-28         0   216         3    11        17   
17772   1216547    2020  2020-09-28         0   571         1    11        17   
17773   1216547    2020  2020-09-28         0   426         1    11        17   
17774   1216547    2020  2020-09-28         0   474         1    11        17   
17775   1216547    2020  2020-09-28         0    20         2    17        11   

       Venue  TossWinner  .

In [3]:
df=pd.read_csv("merged_final_encoded.csv")

# df.to_csv("merged_final_encoded.csv", index=False)

In [20]:
X = df.drop('total_points', axis=1)
y = df.total_points
Z = df

In [22]:
Z_train,Z_test = train_test_split(Z, test_size = 0.3, random_state = 0)
print(Z_train.shape)
print(Z_test.shape)

(13690, 63)
(5868, 63)


In [ ]:
import seaborn as sns
plt.figure(figsize = (70,65))
cor = Z_train.corr()
sns.heatmap(cor, annot=True, cmap = plt.cm.Greens)
plt.show()

In [8]:
X_train.corr()

,Match_Id,Season,Match Num,PlayOffs,Name,Position,Team,Opponent,Venue,TossWinner,...,Sixes_Roll_3_mean,Wd_Roll_3_mean,Nb_Roll_3_mean,Balls_bowled_Roll_3_mean,Catches_Roll_3_mean,total_points_Roll_3_mean,Runs_Roll_3_max,SR_Roll_3_max,Runs_conceded_Roll_3_min,total_points_Roll_3_mean.1
Match_Id,1.000000,0.986990,0.053210,0.080379,0.005139,0.292758,0.150293,0.136712,0.096567,0.134093,...,0.124126,0.021639,-0.035545,0.009356,0.110428,0.061971,0.035843,0.058588,0.014155,0.061971
Season,0.986990,1.000000,0.065690,0.075852,0.004325,0.302669,0.148414,0.134145,0.090072,0.135366,...,0.119603,0.024267,-0.035291,0.011979,0.102684,0.060255,0.033309,0.056750,0.012679,0.060255
Match Num,0.053210,0.065690,1.000000,0.480322,-0.012636,0.040540,0.013170,0.007024,0.045440,-0.028942,...,-0.005843,0.008977,-0.024646,0.007281,0.017434,0.031088,0.004213,0.004507,-0.001255,0.031088
PlayOffs,0.080379,0.075852,0.480322,1.000000,0.003151,0.037219,0.007398,0.006151,-0.025511,-0.035995,...,0.004955,0.011849,-0.023761,0.018320,0.030081,0.035148,0.001885,0.004019,-0.016801,0.035148
Name,0.005139,0.004325,-0.012636,0.003151,1.000000,0.056222,0.007932,-0.010983,-0.037742,-0.008816,...,0.000763,0.005525,-0.004415,-0.002854,-0.010423,-0.001575,-0.001588,-0.017071,-0.016838,-0.001575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
total_points_Roll_3_mean,0.061971,0.060255,0.031088,0.035148,-0.001575,0.042944,0.002596,-0.013902,0.000150,-0.028395,...,-0.059921,0.024468,-0.006535,0.070053,0.290436,1.000000,0.568726,0.262896,0.256539,1.000000
Runs_Roll_3_max,0.035843,0.033309,0.004213,0.001885,-0.001588,0.066503,-0.001707,-0.003329,0.015097,-0.013747,...,-0.253036,-0.235015,-0.104048,-0.457572,0.119977,0.568726,1.000000,0.493979,0.336842,0.568726
SR_Roll_3_max,0.058588,0.056750,0.004507,0.004019,-0.017071,0.028606,-0.000088,-0.005001,0.026897,-0.007687,...,-0.127495,-0.159382,-0.089551,-0.324761,0.097002,0.262896,0.493979,1.000000,0.337853,0.262896
Runs_conceded_Roll_3_min,0.014155,0.012679,-0.001255,-0.016801,-0.016838,0.026210,-0.001651,-0.002562,0.022340,-0.011933,...,-0.208345,-0.220277,-0.079459,-0.390691,0.107422,0.256539,0.336842,0.337853,1.000000,0.256539


In [24]:
 def correlation(dataset, threshold):
        col_corr = set()
        corr_matrix = dataset.corr()
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if abs(corr_matrix.iloc[i,j] > threshold):
                    colname = corr_matrix.columns[i]
                    col_corr.add(colname)
        return col_corr

In [32]:
corr_features = correlation(Z_train, 0.9)
print(len(set(corr_features)))
print(corr_features)

12
{'Runs_Roll_3_max', 'Balls_bowled_Roll_3_mean', 'Dots', 'bowling_points', 'batting_points', 'Balls', 'Balls_bowled', 'Season', 'Balls_Roll_3_mean', 'total_points_Roll_3_mean.1', 'Runs_conceded', 'Consistency'}


In [6]:
# Dots - Bowls bowled
# batting_points* - Runs
# bowling_points - Wickets
# Drop - 'total_points_Roll_3_mean.1', 'Overs'
# Balls_Roll_3_mean* - Runs_Roll_3_mean


# Model Training

### Random Forest

In [4]:
rf = RandomForestRegressor()

In [4]:
X = df.drop(columns={'Date','total_points'}, axis=1)
y=df['total_points']
X_train, X_test, y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state = 33)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(15524, 55)
(3882, 55)
(15524,)
(3882,)


In [32]:
# Manual Hypertunning
model=RandomForestRegressor(n_estimators=300, max_features='sqrt', min_samples_leaf=10, random_state=100).fit(X_train,y_train)

y_pred = model.predict(X_test)
# Evaluate the performance of the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print('MSE:', mse)
print('MAE:', mae)

MSE: 35.04812499200374
MAE: 2.960608175204492


In [37]:
n_estimators=[int(x) for x in np.linspace(start = 200, stop = 2000, num=10)]
max_features=['auto', 'sqrt', 'log2']
max_depth = [int(x) for x in np.linspace(10,1000,10)]
min_samples_split = [1,3,4,5,7,9]
min_samples_leaf = [1,2,4,6,8]
random_grid = {'n_estimators':n_estimators,
    'max_depth':max_depth,       
    'min_samples_split':min_samples_split,
    'min_samples_leaf':min_samples_leaf,
   'max_features': max_features}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [1, 3, 4, 5, 7, 9], 'min_samples_leaf': [1, 2, 4, 6, 8], 'max_features': ['auto', 'sqrt', 'log2']}


In [38]:
rf_randomcv = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,n_iter=100,cv=3,verbose=2,
                                random_state=100,n_jobs=-1)
rf_randomcv.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


D:\SOFTWARE\ANACONDA\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.96940349 0.99323588 0.973455   0.96740331 0.99128298 0.96679094
 0.95295558 0.97717883 0.9735316  0.97733856 0.99306308 0.96701881
 0.99293216 0.97727489 0.96688395 0.96695687 0.9931795  0.95729092
        nan 0.96974063 0.96743234 0.99125857 0.993212          nan
 0.97334559 0.97003773 0.96998879        nan 0.97019231 0.98990571
        nan 0.97372539        nan 0.96194991 0.96998766 0.9618995
 0.96655835 0.99017144        nan 0.97846115        nan 0.99324256
 0.96893378 0.97606113 0.97026529 0.96666282        nan        nan
 0.96199387 0.99322435 0.96159937        nan 0.97365663        nan
 0.96713279        nan 0.97361243 0.96664701        nan 0.97834214
 0.97368186 0.97850026 0.95706094 0.96869159 0.99224659 0.99125792
 0.97026459 0.96748061 0.95325729        nan 0.95281353 0.99091794
 0.9669331  0.96567974 0.97349176 0.97756913 0.97003405 0.

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [1, 3, 4, 5, 7, 9],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=100, verbose=2)

In [49]:
 rf_randomcv.best_params_

{'n_estimators': 1400,
 'min_samples_split': 4,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 230}

In [63]:
best_random_grid=rf_randomcv.best_estimator_ 

NameError: name 'rf_randomcv' is not defined

In [43]:
y_pred = best_random_grid.predict(X_test)
# Evaluate the performance of the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print('MSE:', mse)
print('MAE:', mae)

MSE: 4.5782606163495485
MAE: 0.5432027556768113


### GridSearch CV

In [50]:
# (max_depth=230, min_samples_split=4, n_estimators=1400)
param_grid = {
    'max_depth':[rf_randomcv.best_params_['max_depth']],
    'min_samples_split':[rf_randomcv.best_params_['min_samples_split'] - 2,
                      rf_randomcv.best_params_['min_samples_split'] - 1,
                      rf_randomcv.best_params_['min_samples_split'],
                      rf_randomcv.best_params_['min_samples_split'] + 1,
                      rf_randomcv.best_params_['min_samples_split'] + 2],
    'n_estimators':[rf_randomcv.best_params_['n_estimators'] - 200, 
                    rf_randomcv.best_params_['n_estimators'] - 100,
                    rf_randomcv.best_params_['n_estimators'],
                    rf_randomcv.best_params_['n_estimators'] + 100,
                    rf_randomcv.best_params_['n_estimators'] + 200]
}
print(param_grid)

{'max_depth': [230], 'min_samples_split': [2, 3, 4, 5, 6], 'n_estimators': [1200, 1300, 1400, 1500, 1600]}


In [51]:
rf = RandomForestRegressor()
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,cv=10,n_jobs=-1,verbose=True)
grid_search.fit(X_train,y_train)

Fitting 10 folds for each of 25 candidates, totalling 250 fits


GridSearchCV(cv=10, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'max_depth': [230],
                         'min_samples_split': [2, 3, 4, 5, 6],
                         'n_estimators': [1200, 1300, 1400, 1500, 1600]},
             verbose=True)

In [3]:
# print(rf_randomcv.best_estimator_)
# best_random_grid=rf_randomcv.best_estimator_
best_random_grid={'n_estimators': 1400,
 'min_samples_split': 4,
 'min_samples_leaf': 1,
 'max_features': 'auto',
 'max_depth': 230}

In [11]:
y_pred = best_random_grid.predict(X_test)
# Evaluate the performance of the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print('MSE:', mse)
print('MAE:', mae)

AttributeError: 'dict' object has no attribute 'predict'

In [5]:
# Training accrding to the best parameters
# {'n_estimators': 1400,
#  'min_samples_split': 4,
#  'min_samples_leaf': 1,
#  'max_features': 'auto',
#  'max_depth': 230}
model=RandomForestRegressor(n_estimators=1400, max_features='auto', min_samples_split=4, max_depth = 230).fit(X_train,y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('RMSE:', rmse )
print('MSE:', mse)
print('MAE:', mae)

RMSE: 2.1328033715005907
MSE: 4.548850221484288
MAE: 0.544302231377648


In [8]:
print(X_test.shape)
print(y_pred.shape)

(3882, 55)
(3882,)


In [34]:
y_pred
df1 = X_test.copy()

### Working with XG Boost

In [ ]:
train_size = int(0.8 * len(df))
train_set = df[:train_size]
test_set = df[train_size:]

In [ ]:
X_train = train_set.drop(['total_points', 'Date'], axis=1)
y_train = train_set['total_points']
X_test = test_set.drop(['total_points', 'Date'], axis=1)
y_test = test_set['total_points']


In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [ ]:
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [ ]:
params = {}
num_rounds = 10
xgb_model = xgb.train(params, dtrain, num_boost_round=num_rounds)

In [ ]:
dtest = xgb.DMatrix(X_test)

In [ ]:
y_pred = xgb_model.predict(dtest)

In [ ]:
# Evaluate the performance of the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print('MSE:', mse)
print('RMSE:', rmse)
print('MAE:', mae)

# MSE: 12.843018746392431
# RMSE: 3.583715773661805
# MAE: 1.6785462360500834

##### Hyperparameter optimization using Random search CV and Grisd Search CV

In [ ]:
params_grid = {
    'learning_rate': [0.1, 0.01],
    'max_depth': [3, 6, 9],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5],
    'reg_lambda': [0.0, 0.1, 0.5]
}


In [ ]:
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)


In [ ]:
random_search = RandomizedSearchCV(xgb_model, param_distributions=params_grid, n_iter=100, cv=5, n_jobs=-1, random_state=42)

random_search.fit(X_train, y_train)

In [ ]:
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
grid_search = GridSearchCV(xgb_model, params_grid, cv=5, scoring=make_scorer(mean_squared_error), verbose=1)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_ = {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 
                            'reg_alpha': 0.5, 'reg_lambda': 0.5, 'subsample': 0.8}

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, **random_search.best_params_)
xgb_model.fit(X_train, y_train)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print('MSE:', mse)
print('RMSE:', rmse)
print('MAE:', mae)

# MSE: 1.2048790411629329
# RMSE: 1.0976698233817548
# MAE: 0.6398210514124709

##### Applying regularization

In [ ]:
# Create the Ridge regression model with alpha=0.5
ridge = Ridge(alpha=0.5)

ridge.fit(X_train, y_train)


In [ ]:
alpha_values = [0.01, 0.1, 1]
# Loop over alpha values and fit Ridge model with each value
for alpha in alpha_values:
    # Initialize Ridge model with current alpha value
    model = Ridge(alpha=alpha)
    
    # Fit Ridge model on training data
    model.fit(X_train, y_train)
    
    # Predict on testing data
    y_pred = model.predict(X_test)
    
    # Calculate MSE and print results
    mse = mean_squared_error(y_test, y_pred)
    print("Alpha:", alpha)
    print("MSE:", mse)

In [ ]:
y_pred = ridge.predict(X_test)


In [ ]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print('MSE:', mse)
print('RMSE:', rmse)
print('MAE:', mae)

# MSE: 4.166307763305432e-09
# RMSE: 6.454694232343955e-05
# MAE: 4.3216927832990696e-05

In [ ]:
import pickle
pickle.dump(model, open('xgb_model.pkl', 'wb'))